In [14]:
import requests
import json
import pandas as pd
from pyproj import Proj, transform

In [2]:
# request headers for the subscription
#headers = {"X-User-id": "6278", "X-User-hash":"a235942bff158e1027d455a9d36d2fa3"}
headers = {"X-User-id": "www", "X-User-hash":"global"}
bv_devices_id = ['82000169', '160000C3', '160000A9', '1600009B', '160000A7'] 

In [3]:
def make_uradmonitor_request(url: str, headers):
    response = requests.get(url, headers = headers)
    try:
        response_json = json.loads(response.content.decode("utf-8"))
    except Exception as e:
        print (e)
        return str(e)

    return response_json
    

In [7]:
def insert(df, row):
    insert_loc = df.index.max()

    if pd.isna(insert_loc):
        df.loc[0] = row
    else:
        df.loc[insert_loc + 1] = row

In [10]:
def LatLon_to_EN(lat, lon):
    try:
      easting, northing = transform( Proj('epsg:4326'), Proj('epsg:3857'), lat, lon) # from WGS-84 to Web Mercator Easting/Northing
      return easting, northing
    except:
      return None, None

### 1. Get devices list

In [4]:
devices_data = make_uradmonitor_request('https://data.uradmonitor.com/api/v1/devices/', headers = headers)

In [6]:
brasov_devices = [device for device in devices_data if device['id'] in bv_devices_id]
pm1 =  [{'pm1': brasov_device['avg_pm1'], 'latitude':brasov_device['latitude'], 'longitude':brasov_device['longitude'], 'avg_temperature': brasov_device['avg_temperature']} for brasov_device in brasov_devices]
pm25 = [{'pm25': brasov_device['avg_pm25'], 'latitude':brasov_device['latitude'], 'longitude':brasov_device['longitude']} for brasov_device in brasov_devices]
pm10 = [{'pm10': brasov_device['avg_pm10'], 'latitude':brasov_device['latitude'], 'longitude':brasov_device['longitude']} for brasov_device in brasov_devices]

In [15]:
bv_sensor_data = pd.DataFrame(columns = ['deviceid', 'pm1', 'pm25', 'pm10', 'latitude', 'longitude', 'eastern', 'northern', 'temperature'])
for idx, device in enumerate(devices_data):
    if device['id'] in bv_devices_id:
        E, N = LatLon_to_EN(device['latitude'], device['longitude'])
        insert(bv_sensor_data, [device['id'], int(device['avg_pm1']), int(device['avg_pm25']), int(device['avg_pm10']), device['latitude'], device['longitude'], E, N, device['avg_temperature']])


In [16]:
bv_sensor_data

,deviceid,pm1,pm25,pm10,latitude,longitude,eastern,northern,temperature
0,1600009B,18,31,34,45.653509,25.566120,2.846007e+06,5.724997e+06,8.56
1,160000A7,20,46,52,45.704032,25.640955,2.854338e+06,5.733046e+06,7.58
2,160000A9,13,35,43,45.656800,25.591700,2.848855e+06,5.725521e+06,8.44
3,82000169,13,17,19,45.645893,25.599471,2.849720e+06,5.723784e+06,11.03


### 2. Get device sensors list

In [32]:
devices_id = [device['id'] for device in devices_list ]

In [34]:
available_sensors = [make_uradmonitor_request(f'https://data.uradmonitor.com/api/v1/devices/{id}', headers = headers) for id in devices_id]

### 3. Get detailed device data

In [ ]:
start_interval = 600
stop_interval = 480
sensor_values = dict()
for device_id in devices_id:
    sensor_data = [make_uradmonitor_request(f'https://data.uradmonitor.com/api/v1/devices/{device_id}/{available_sensors data for ]
    sensor_data = make_uradmonitor_request(f'https://data.uradmonitor.com/api/v1/devices/{device_id}/{available_sensors[0]["temperature"]}/{start_interval}/{stop_interval}',headers = headers)
sensor_data

In [ ]:
for key in available_sensors[0].keys():
    print(key)

In [ ]:
available_sensors

In [97]:
sensor_data_global = dict()
for device_id in devices_id:
    for sensor in available_sensors:
        sensor_data = [{sensor_type: make_uradmonitor_request(f'https://data.uradmonitor.com/api/v1/devices/{device_id}/{sensor_type}/{start_interval}', headers=headers)} for sensor_type in sensor.keys()]
    sensor_data_global.update({device_id:sensor_data})

In [ ]:
sensor_data_global

In [46]:
import urllib.request
opener = urllib.request.build_opener()
opener.addheaders = [("X-User-id", "6378"), ("X-User-hash", "e3e7cbb6623b3e6a4504a2a258da07f1")]
devices = opener.open('https://data.uradmonitor.com/api/v1/devices')

In [47]:
import json
js = json.loads(f"{devices.read().decode('utf-8')}")

In [48]:
json.dumps(js[0])

'{"id": "82000156", "timefirst": "1580763711", "timelast": "1581573784", "timelocal": "240", "latitude": 45.700884, "longitude": 26.1063, "altitude": 470.5, "speed": 0, "city": "Harman", "country": "RO", "placement": "1", "versionsw": "74", "versionhw": "108", "status": null, "mobile": null, "detector": "SI29BG", "factor": 0.01, "note": "", "picture": "", "avg_timelocal": "750", "avg_temperature": "21.94", "avg_pressure": "95778", "avg_humidity": "43.91", "avg_voc": "110909", "min_voc": null, "max_voc": null, "avg_noise": "53.05", "avg_co2": "1103", "avg_ch2o": "101", "avg_o3": "20", "avg_pm1": "0", "avg_pm25": "0", "avg_pm10": "0"}'

In [22]:
import json
import urllib.request
from pyproj import Proj, transform

In [41]:
from bokeh.plotting import figure, show, output_file, output_notebook, ColumnDataSource
from bokeh.models import ColorBar
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models.tools import WheelZoomTool, HoverTool
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6, Viridis256
import pandas as pd

bv_devices_id = ['82000169', '160000C3', '160000A9', '1600009B', '160000A7'] 
static_sensor_data = [{'deviceid': '1600009B',
  'latitude': 45.653509,
  'longitude': 25.56612,
  'eastern': 2846007.4599597272,
  'northern': 5724996.5941273},
 {'deviceid': '160000A7',
  'latitude': 45.704032,
  'longitude': 25.640955,
  'eastern': 2854338.0540532423,
  'northern': 5733046.333381937},
 {'deviceid': '160000A9',
  'latitude': 45.6568,
  'longitude': 25.5917,
  'eastern': 2848855.0125342193,
  'northern': 5725520.722051452},
 {'deviceid': '160000C3',
  'latitude': 45.6536,
  'longitude': 25.599,
  'eastern': 2849667.64481701,
  'northern': 5725011.086463493},
 {'deviceid': '82000169',
  'latitude': 45.645893,
  'longitude': 25.599471,
  'eastern': 2849720.0762971737,
  'northern': 5723783.780513774}]

opener = urllib.request.build_opener()
opener.addheaders = [("X-User-id", "www"), ("X-User-hash", "global")]
devices = opener.open('https://data.uradmonitor.com/api/v1/devices')

devices_list = json.loads(f'{devices.read().decode("utf-8")}')
bv_sensor_data = pd.DataFrame(columns = ['deviceid', 'pm1', 'pm25', 'pm10', 'latitude', 'longitude', 'eastern', 'northern', 'temperature'])
for device in devices_list:
    if device['id'] in bv_devices_id:
        E, N = LatLon_to_EN(device['latitude'], device['longitude'])
        insert(bv_sensor_data, [device['id'], int(device['avg_pm1']), int(device['avg_pm25']), int(device['avg_pm10']), device['latitude'], device['longitude'], E, N, device['avg_temperature']])
        #for sensor_data in static_sensor_data:
        #    sensor_data.update({'pm1': int(device['avg_pm1']), 'pm25': int(device['avg_pm25']), 'pm10': int(device['avg_pm10']), 'temperature': device['avg_temperature']})

#json.dumps(static_sensor_data)
                          
tile_provider = get_provider(Vendors.STAMEN_TERRAIN)

x_range=(2839667, 2864387)
y_range=(5724182, 5733344)

#FF0000	Bad
#FFFF00	Bad-Average
#FFFF00	Average
#7FFF00	Average-Good
#00FF00                          
#custom_pallette = ['#2E7F18', '#45731E', '#675E24', '#8D472B', '#B13433', '#C82538']
custom_pallette = ['#00FF00', '#7FFF00', '#FFFF00', '#FFFF00', '#FF0000', '#C82538']
#Vendors.
# range bounds supplied in web mercator coordinates
p = figure(tools='pan,wheel_zoom,reset', plot_width=1200, plot_height=900, x_range=x_range, y_range=y_range,x_axis_type="mercator", y_axis_type="mercator")

p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.add_tile(tile_provider)
p.toolbar.active_scroll = p.select_one(WheelZoomTool)

#source = ColumnDataSource(pd.DataFrame(static_sensor_data))
source = ColumnDataSource(bv_sensor_data)
mapper = linear_cmap(field_name='pm25', palette=custom_pallette ,low=float(1) ,high=float(130))
p.circle(x='eastern', y='northern', size='pm25', alpha=0.5, line_color=mapper, color=mapper, hover_color="navy", source=source)
color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0), title='PM2.5')

p.add_layout(color_bar, 'right')

p.add_tools(HoverTool(
    tooltips=[
        ( 'pm1',   '@pm1' ),
        ( 'pm2.5',  '@pm25'), # use @{ } for field names with spaces
        ( 'pm10', '@pm10'      )
    ],    

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
))    
output_notebook()
#output_file("index_2.html", title = 'Brasov Air Pollution')

show(p)

Loading BokehJS ...

In [29]:
panda= pd.DataFrame(static_sensor_data)
#source = ColumnDataSource(bv_sensor_data)
panda

,deviceid,eastern,latitude,longitude,northern,pm1,pm10,pm25,temperature
0,1600009B,2846007.4599597272,45.653509,25.566120,5724996.5941273,14,21,18,10.94
1,160000A7,2854338.0540532423,45.704032,25.640955,5733046.333381937,14,21,18,10.94
2,160000A9,2848855.0125342193,45.656800,25.591700,5725520.722051452,14,21,18,10.94
3,160000C3,2849667.64481701,45.653600,25.599000,5725011.086463493,14,21,18,10.94
4,82000169,2849720.0762971737,45.645893,25.599471,5723783.780513774,14,21,18,10.94


In [4]:
x_range=(2839667, 2864387)
y_range=(5724182, 5733344)

In [5]:
x_range

(2819500, 2859900)

In [6]:
y_range

(5723000, 5733000)

In [4]:
def heatmap(d, bins=(100,100), smoothing=1.3, cmap='jet'):
    def getx(pt):
        return pt.coords[0][0]

    def gety(pt):
        return pt.coords[0][1]

    x = list(d.geometry.apply(getx))
    y = list(d.geometry.apply(gety))
    heatmap, xedges, yedges = np.histogram2d(y, x, bins=bins)
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]

    logheatmap = np.log(heatmap)
    logheatmap[np.isneginf(logheatmap)] = 0
    logheatmap = ndimage.filters.gaussian_filter(logheatmap, smoothing, mode='nearest')
    
    plt.imshow(logheatmap, cmap=cmap, extent=extent)
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
pts = pd.DataFrame([1,2,3,4,5,6,7,8] , columns = ['pla'])
heatmap(pts, bins=50, smoothing=1.5)

In [89]:
list(bv_sensor_data['pm10'])

['25', '47', '46', '42', '17']

In [82]:
list(zip(bv_sensor_data['latitude'], bv_sensor_data['longitude']))

[(45.653509, 25.56612),
 (45.704032, 25.640955),
 (45.6568, 25.5917),
 (45.6486, 25.6061),
 (45.645893, 25.599471)]

In [88]:
list(bv_sensor_data['pm25'])

['25', '40', '36', '37', '15']

In [94]:
bv_sensor_data.to_json(orient='records')

'[{"deviceid":"1600009B","pm1":"16","pm25":"25","pm10":"25","latitude":45.653509,"longitude":25.56612,"temperature":"18.69"},{"deviceid":"160000A7","pm1":"18","pm25":"40","pm10":"47","latitude":45.704032,"longitude":25.640955,"temperature":"13.28"},{"deviceid":"160000A9","pm1":"16","pm25":"36","pm10":"46","latitude":45.6568,"longitude":25.5917,"temperature":"12.79"},{"deviceid":"160000C3","pm1":"19","pm25":"37","pm10":"42","latitude":45.6486,"longitude":25.6061,"temperature":"13.03"},{"deviceid":"82000169","pm1":"12","pm25":"15","pm10":"17","latitude":45.645893,"longitude":25.599471,"temperature":"19.01"}]'

In [129]:
bv_sensor_data

,deviceid,pm1,pm25,pm10,latitude,longitude,eastern,northern,temperature
0,1600009B,16,25,25,45.653509,25.566120,2.846007e+06,5.724997e+06,18.69
1,160000A7,18,40,47,45.704032,25.640955,2.854338e+06,5.733046e+06,13.28
2,160000A9,16,36,46,45.656800,25.591700,2.848855e+06,5.725521e+06,12.79
3,160000C3,19,37,42,45.648600,25.606100,2.850458e+06,5.724215e+06,13.03
4,82000169,12,15,17,45.645893,25.599471,2.849720e+06,5.723784e+06,19.01


In [121]:
Spectral6

['#3288bd', '#99d594', '#e6f598', '#fee08b', '#fc8d59', '#d53e4f']

In [154]:
dic = bv_sensor_data.to_dict(orient='records')

In [156]:
type(dic)

list